# 括弧用擬似検出データ生成

## このノートブックの目的

Kaggleの「Handwritten math symbols dataset（xainano）」から括弧 `(` と `)` の単体画像を取得し、
擬似検出データ（YOLO形式）を生成します。

### 処理の流れ
1. Kaggleからデータセットをダウンロード
2. 括弧 `(` と `)` の単体画像を抽出
3. 各画像から黒画素の外接矩形を計算（bbox作成）
4. その記号を640x640の白キャンバスに貼る（位置と大きさをランダム化）
5. 貼った位置・サイズから正しいbboxをYOLO形式で保存
6. 既存のAidaデータセットと混ぜる準備（tar化・S3アップロード）

### 出力
- `/content/tmp/paren_synthetic/train/images/` - 生成された画像
- `/content/tmp/paren_synthetic/train/labels/` - YOLO形式のラベル
- `/content/tmp/paren_synthetic/paren_synthetic.tar` - tarアーカイブ（S3アップロード用）

### 注意点
- クラスID: `(` は42、`)` は43（既存のYOLOモデルと一致）
- 生成枚数: 各1500〜3000枚（合計3000〜6000枚）
- 画像サイズ: 640x640（YOLO学習用）


## セル0: AWS認証設定


In [ ]:
import os

os.environ["AWS_ACCESS_KEY_ID"] = "AKIAxxxxxxxxxxxxxxxx"  # 埋めてね！！！！！！！
os.environ["AWS_SECRET_ACCESS_KEY"] = "xxxxxxxxxxxxxxxxxxxx"  # 埋めてね！！！！！！！
os.environ["AWS_DEFAULT_REGION"] = "ap-northeast-1"

print("AWS env set")

assert "AWS_ACCESS_KEY_ID" in os.environ, "AWS_ACCESS_KEY_ID not set"
assert "AWS_SECRET_ACCESS_KEY" in os.environ, "AWS_SECRET_ACCESS_KEY not set"


## セル0.5: Kaggle認証設定


In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = 'masapirika'
os.environ['KAGGLE_KEY'] = '================'


## セル1: 設定（ここだけ触ればOK）


In [ ]:
# ============================================
# 設定（ここだけ変更すればOK）
# ============================================

import os
from pathlib import Path

# ===== Kaggle設定 =====
KAGGLE_DATASET = "xainano/handwrittenmathsymbols"  # 括弧用データセット
KAGGLE_DOWNLOAD_DIR = Path("/content/tmp/kaggle_paren")

# ===== 出力設定 =====
OUT_DIR = Path("/content/tmp/paren_synthetic")
TRAIN_IMAGES_DIR = OUT_DIR / "train" / "images"
TRAIN_LABELS_DIR = OUT_DIR / "train" / "labels"

# ===== 生成設定 =====
# クラスID（既存のYOLOモデルと一致）
CLASS_ID_LPAREN = 42  # (
CLASS_ID_RPAREN = 43  # )

# 生成枚数（各クラス）
NUM_SAMPLES_LPAREN = 2500  # ( の生成枚数（1500〜3000の範囲で調整）
NUM_SAMPLES_RPAREN = 2500  # ) の生成枚数（1500〜3000の範囲で調整）

# 画像サイズ（YOLO学習用）
CANVAS_SIZE = 640  # 640x640

# 貼り付け時のランダム化パラメータ
MIN_SCALE = 0.5  # 最小スケール（元画像に対する）← 0.3から0.5に変更（bboxが小さすぎるのを防ぐ）
MAX_SCALE = 1.2  # 最大スケール（元画像に対する）← 0.8から1.2に変更（より大きめに）
MIN_MARGIN = 20  # 最小マージン（ピクセル）
MAX_MARGIN = 100  # 最大マージン（ピクセル）
# 注意: bboxが画像の5%以上（640x640なら32x32ピクセル以上）になるように調整
MIN_BBOX_SIZE = 32  # 最小bboxサイズ（ピクセル）

# ===== S3設定 =====
S3_BUCKET = "km62m-ml-storage"  # S3バケット名
S3_PREFIX = "yolo-dataset/finetune2-paren"  # S3プレフィックス
UPLOAD_TO_S3 = True  # TrueにするとS3へアップロード

# ===== その他 =====
SEED = 42  # 再現性のためのシード
CLEAN_OUT_DIR = False  # Trueにすると既存のOUT_DIRを削除
FORCE_REBUILD_TAR = False  # Trueにすると既存のtarファイルを再作成（S3アップロード前に再生成したい場合）
FORCE_UPLOAD_S3 = True  # Trueにすると既存のS3ファイルを上書き、Falseにするとスキップ

print("✓ 設定完了")
print(f"  KAGGLE_DATASET: {KAGGLE_DATASET}")
print(f"  OUT_DIR: {OUT_DIR}")
print(f"  NUM_SAMPLES_LPAREN: {NUM_SAMPLES_LPAREN}")
print(f"  NUM_SAMPLES_RPAREN: {NUM_SAMPLES_RPAREN}")
print(f"  CANVAS_SIZE: {CANVAS_SIZE}x{CANVAS_SIZE}")


## セル2: 依存パッケージのインストール


In [ ]:
# ============================================
# 依存パッケージのインストール
# ============================================

import subprocess
import sys

def install_package(package, import_name=None):
    """
    パッケージをインストール（必要に応じて）
    """
    if import_name is None:
        import_name = package
    
    try:
        __import__(import_name)
        print(f"✓ {package} は既にインストール済み")
    except ImportError:
        print(f"📦 {package} をインストール中...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"✓ {package} インストール完了")

# 必要なパッケージ
packages = [
    ("kaggle", None),
    ("Pillow", "PIL"),
    ("numpy", None),
    ("tqdm", None),
]

for pkg_info in packages:
    if isinstance(pkg_info, tuple):
        pkg, import_name = pkg_info
    else:
        pkg, import_name = pkg_info, None
    install_package(pkg, import_name)

# AWS CLI
try:
    result = subprocess.run(["aws", "--version"], capture_output=True, text=True)
    print(f"\n✓ AWS CLI は既にインストール済み: {result.stdout.strip()}")
except FileNotFoundError:
    print("\n📦 AWS CLI をインストール中...")
    subprocess.check_call(["pip", "install", "-q", "awscli"])
    print("✓ AWS CLI インストール完了")

print("\n✓ 全ての依存パッケージの準備完了")


## セル3: Kaggleからデータセットをダウンロード


In [ ]:
# ============================================
# Kaggleからデータセットをダウンロード
# ============================================

import subprocess
from pathlib import Path

print("=" * 80)
print("Kaggleからデータセットをダウンロード")
print("=" * 80)

# Kaggle認証の確認
has_env = bool(os.getenv("KAGGLE_USERNAME")) and bool(os.getenv("KAGGLE_KEY"))
has_json = (Path.home() / ".config" / "kaggle" / "kaggle.json").exists() or (Path.home() / ".kaggle" / "kaggle.json").exists()

if not (has_env or has_json):
    print("⚠️ Kaggle認証が見つかりません")
    print("  セル0.5でKAGGLE_USERNAMEとKAGGLE_KEYを設定してください")
    raise ValueError("Kaggle認証が必要です")
else:
    print("✓ Kaggle認証OK")

# ダウンロード先の準備
KAGGLE_DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)

# Kaggle CLIでダウンロード
print(f"\n📥 KaggleからDL中: {KAGGLE_DATASET}")
try:
    subprocess.run(
        ["kaggle", "datasets", "download", "-d", KAGGLE_DATASET, "-p", str(KAGGLE_DOWNLOAD_DIR)],
        check=True
    )
    print("✓ ダウンロード完了")
except subprocess.CalledProcessError as e:
    print(f"❌ ダウンロード失敗: {e}")
    raise

# zipファイルを確認
zip_files = sorted(KAGGLE_DOWNLOAD_DIR.glob("*.zip"), key=lambda p: p.stat().st_mtime, reverse=True)
if zip_files:
    zip_path = zip_files[0]
    size_mb = zip_path.stat().st_size / (1024 ** 2)
    print(f"\n✓ zipファイルを確認: {zip_path.name} ({size_mb:.2f} MB)")
else:
    raise FileNotFoundError("zipファイルが見つかりませんでした")


## セル4: zipファイルを展開して括弧画像を抽出


In [ ]:
# ============================================
# zipファイルを展開して括弧画像を抽出
# ============================================

import zipfile
from pathlib import Path
from PIL import Image
import numpy as np
from tqdm import tqdm

print("=" * 80)
print("zipファイルを展開して括弧画像を抽出")
print("=" * 80)

# zipファイルのパス
zip_files = sorted(KAGGLE_DOWNLOAD_DIR.glob("*.zip"), key=lambda p: p.stat().st_mtime, reverse=True)
if not zip_files:
    raise FileNotFoundError("zipファイルが見つかりません")
zip_path = zip_files[0]

# 展開先
EXTRACT_DIR = KAGGLE_DOWNLOAD_DIR / "extracted"
EXTRACT_DIR.mkdir(parents=True, exist_ok=True)

# zipファイルを展開
print(f"\n📦 zipファイルを展開中: {zip_path.name}")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    for file_info in tqdm(file_list, desc="展開中"):
        zip_ref.extract(file_info, EXTRACT_DIR)

print(f"\n✓ 展開完了: {EXTRACT_DIR}")

# 括弧画像を探す（ファイル名やフォルダ構造に応じて調整が必要）
print(f"\n🔍 括弧画像を検索中...")

# 画像ファイルを再帰的に検索
image_extensions = {'.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG'}
all_images = []
for ext in image_extensions:
    all_images.extend(EXTRACT_DIR.rglob(f"*{ext}"))

print(f"  見つかった画像ファイル: {len(all_images)} 件")

# 括弧画像を分類（ファイル名やパスから判定）
lparen_images = []
rparen_images = []

for img_path in all_images:
    path_str = str(img_path).lower()
    name = img_path.stem.lower()
    
    # 左括弧のパターン
    if any(pattern in path_str or pattern in name for pattern in ['lparen', 'left_paren', 'leftparen', '(', 'open_paren']):
        lparen_images.append(img_path)
    # 右括弧のパターン
    elif any(pattern in path_str or pattern in name for pattern in ['rparen', 'right_paren', 'rightparen', ')', 'close_paren']):
        rparen_images.append(img_path)

print(f"\n✓ 括弧画像を分類")
print(f"  ( の画像: {len(lparen_images)} 件")
print(f"  ) の画像: {len(rparen_images)} 件")

if len(lparen_images) == 0 and len(rparen_images) == 0:
    print("\n⚠️  警告: 括弧画像が見つかりませんでした")
    print("  ファイル構造を確認してください")
    print("\n見つかった画像の例（最初の10件）:")
    for img_path in all_images[:10]:
        print(f"    {img_path.relative_to(EXTRACT_DIR)}")

print(f"\n✓ 画像抽出完了")


In [ ]:
# ============================================
# 擬似検出データを生成
# ============================================

import random
import numpy as np
from PIL import Image, ImageDraw
from pathlib import Path
from tqdm import tqdm
import uuid
import shutil

print("=" * 80)
print("擬似検出データを生成")
print("=" * 80)

# シードを設定
random.seed(SEED)
np.random.seed(SEED)

# 出力ディレクトリの準備
if CLEAN_OUT_DIR and OUT_DIR.exists():
    print(f"🗑️  既存のOUT_DIRを削除: {OUT_DIR}")
    shutil.rmtree(OUT_DIR)

TRAIN_IMAGES_DIR.mkdir(parents=True, exist_ok=True)
TRAIN_LABELS_DIR.mkdir(parents=True, exist_ok=True)

def get_bounding_box(image: Image.Image) -> tuple:
    """
    画像から黒画素の外接矩形を計算
    戻り値: (xmin, ymin, xmax, ymax) ピクセル座標
    """
    # グレースケールに変換
    if image.mode != 'L':
        image = image.convert('L')
    
    # numpy配列に変換
    img_array = np.array(image)
    
    # 黒画素（値が小さい画素）を探す
    # 閾値: 128以下を黒とみなす（必要に応じて調整）
    black_pixels = np.where(img_array < 128)
    
    if len(black_pixels[0]) == 0:
        # 黒画素が見つからない場合は画像全体を返す
        return (0, 0, image.width, image.height)
    
    ymin = int(np.min(black_pixels[0]))
    ymax = int(np.max(black_pixels[0]))
    xmin = int(np.min(black_pixels[1]))
    xmax = int(np.max(black_pixels[1]))
    
    # 少しマージンを追加（余白を確保）
    margin = 2
    xmin = max(0, xmin - margin)
    ymin = max(0, ymin - margin)
    xmax = min(image.width, xmax + margin)
    ymax = min(image.height, ymax + margin)
    
    return (xmin, ymin, xmax, ymax)

def create_synthetic_image(source_image: Image.Image, class_id: int, uuid_str: str) -> tuple:
    """
    単体画像から擬似検出画像を生成
    戻り値: (生成画像, YOLO形式bbox)
    """
    # 外接矩形を計算
    xmin_src, ymin_src, xmax_src, ymax_src = get_bounding_box(source_image)
    
    # 外接矩形のサイズ
    bbox_width = xmax_src - xmin_src
    bbox_height = ymax_src - ymin_src
    
    if bbox_width <= 0 or bbox_height <= 0:
        # 無効なbboxの場合は画像全体を使用
        bbox_width = source_image.width
        bbox_height = source_image.height
        xmin_src, ymin_src = 0, 0
    
    # スケールをランダムに決定
    scale = random.uniform(MIN_SCALE, MAX_SCALE)
    
    # 新しいサイズを計算（アスペクト比を保持）
    new_width = int(bbox_width * scale)
    new_height = int(bbox_height * scale)
    
    # 最小サイズを確保（YOLO学習に適したサイズ）
    # 640x640の画像に対して、bboxは5%以上（32x32ピクセル以上）が推奨
    new_width = max(MIN_BBOX_SIZE, new_width)
    new_height = max(MIN_BBOX_SIZE, new_height)
    
    # 外接矩形部分を切り出してリサイズ
    cropped = source_image.crop((xmin_src, ymin_src, xmax_src, ymax_src))
    resized = cropped.resize((new_width, new_height), Image.Resampling.LANCZOS)
    
    # 640x640の白キャンバスを作成
    canvas = Image.new('RGB', (CANVAS_SIZE, CANVAS_SIZE), color='white')
    
    # 貼り付け位置をランダムに決定（マージンを考慮）
    margin_x = random.randint(MIN_MARGIN, MAX_MARGIN)
    margin_y = random.randint(MIN_MARGIN, MAX_MARGIN)
    
    # 貼り付け位置（左上座標）
    paste_x = margin_x
    paste_y = margin_y
    
    # キャンバスからはみ出さないように調整
    paste_x = min(paste_x, CANVAS_SIZE - new_width - 10)
    paste_y = min(paste_y, CANVAS_SIZE - new_height - 10)
    paste_x = max(0, paste_x)
    paste_y = max(0, paste_y)
    
    # 画像を貼り付け
    canvas.paste(resized, (paste_x, paste_y))
    
    # YOLO形式のbboxを計算（正規化座標）
    center_x = (paste_x + new_width / 2) / CANVAS_SIZE
    center_y = (paste_y + new_height / 2) / CANVAS_SIZE
    width_norm = new_width / CANVAS_SIZE
    height_norm = new_height / CANVAS_SIZE
    
    # 0-1の範囲にクリップ
    center_x = max(0.0, min(1.0, center_x))
    center_y = max(0.0, min(1.0, center_y))
    width_norm = max(0.0, min(1.0, width_norm))
    height_norm = max(0.0, min(1.0, height_norm))
    
    return canvas, (class_id, center_x, center_y, width_norm, height_norm)

# 左括弧 `(` を生成
print(f"\n📝 左括弧 `(` の擬似検出データを生成中...")
if len(lparen_images) == 0:
    print("⚠️  警告: 左括弧の画像が見つかりませんでした")
    print("  スキップします")
else:
    generated_count = 0
    for i in tqdm(range(NUM_SAMPLES_LPAREN), desc="生成中"):
        # ランダムに元画像を選択
        source_img_path = random.choice(lparen_images)
        
        try:
            # 画像を読み込み
            source_img = Image.open(source_img_path)
            
            # UUIDを生成
            uuid_str = f"lparen_{uuid.uuid4().hex[:8]}_{i:06d}"
            
            # 擬似検出画像を生成
            synthetic_img, yolo_bbox = create_synthetic_image(source_img, CLASS_ID_LPAREN, uuid_str)
            
            # 画像を保存
            img_path = TRAIN_IMAGES_DIR / f"{uuid_str}.png"
            synthetic_img.save(img_path)
            
            # ラベルを保存
            label_path = TRAIN_LABELS_DIR / f"{uuid_str}.txt"
            with open(label_path, 'w') as f:
                class_id, cx, cy, w, h = yolo_bbox
                f.write(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")
            
            generated_count += 1
        except Exception as e:
            print(f"\n⚠️  警告: {source_img_path} の処理に失敗: {e}")
            continue
    
    print(f"\n✓ 左括弧 `(` の生成完了: {generated_count} 件")

# 右括弧 `)` を生成
print(f"\n📝 右括弧 `)` の擬似検出データを生成中...")
if len(rparen_images) == 0:
    print("⚠️  警告: 右括弧の画像が見つかりませんでした")
    print("  スキップします")
else:
    generated_count = 0
    for i in tqdm(range(NUM_SAMPLES_RPAREN), desc="生成中"):
        # ランダムに元画像を選択
        source_img_path = random.choice(rparen_images)
        
        try:
            # 画像を読み込み
            source_img = Image.open(source_img_path)
            
            # UUIDを生成
            uuid_str = f"rparen_{uuid.uuid4().hex[:8]}_{i:06d}"
            
            # 擬似検出画像を生成
            synthetic_img, yolo_bbox = create_synthetic_image(source_img, CLASS_ID_RPAREN, uuid_str)
            
            # 画像を保存
            img_path = TRAIN_IMAGES_DIR / f"{uuid_str}.png"
            synthetic_img.save(img_path)
            
            # ラベルを保存
            label_path = TRAIN_LABELS_DIR / f"{uuid_str}.txt"
            with open(label_path, 'w') as f:
                class_id, cx, cy, w, h = yolo_bbox
                f.write(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")
            
            generated_count += 1
        except Exception as e:
            print(f"\n⚠️  警告: {source_img_path} の処理に失敗: {e}")
            continue
    
    print(f"\n✓ 右括弧 `)` の生成完了: {generated_count} 件")

# 生成結果の確認
generated_images = list(TRAIN_IMAGES_DIR.glob("*.png"))
generated_labels = list(TRAIN_LABELS_DIR.glob("*.txt"))

print(f"\n" + "=" * 80)
print("生成結果")
print("=" * 80)
print(f"  生成画像数: {len(generated_images)} 件")
print(f"  生成ラベル数: {len(generated_labels)} 件")

# 1対1対応の確認
image_uuids = {f.stem for f in generated_images}
label_uuids = {f.stem for f in generated_labels}
matched = image_uuids & label_uuids

print(f"\n1対1対応確認:")
print(f"  マッチしたペア: {len(matched)}/{len(generated_images)}")

if len(matched) == len(generated_images) == len(generated_labels):
    print(f"\n✓ 完璧！全ての画像とラベルが1対1で対応しています。")
else:
    print(f"\n⚠️  警告: 一部のファイルが対応していません")

print(f"\n✓ 擬似検出データ生成完了")


## セル5.5: bboxサイズの統計確認（重要）


In [ ]:
# ============================================
# bboxサイズの統計確認（重要）
# ============================================

import numpy as np

print("=" * 80)
print("bboxサイズの統計確認")
print("=" * 80)

bbox_widths = []
bbox_heights = []
bbox_areas = []

for label_path in TRAIN_LABELS_DIR.glob("*.txt"):
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                width_norm = float(parts[3])
                height_norm = float(parts[4])
                
                # ピクセルサイズに変換
                width_px = width_norm * CANVAS_SIZE
                height_px = height_norm * CANVAS_SIZE
                area_px = width_px * height_px
                
                bbox_widths.append(width_px)
                bbox_heights.append(height_px)
                bbox_areas.append(area_px)

if len(bbox_widths) > 0:
    print(f"\nbboxサイズ統計（ピクセル単位、画像サイズ={CANVAS_SIZE}x{CANVAS_SIZE}）:")
    print(f"  幅:")
    print(f"    平均: {np.mean(bbox_widths):.1f} px")
    print(f"    最小: {np.min(bbox_widths):.1f} px")
    print(f"    最大: {np.max(bbox_widths):.1f} px")
    print(f"  高さ:")
    print(f"    平均: {np.mean(bbox_heights):.1f} px")
    print(f"    最小: {np.min(bbox_heights):.1f} px")
    print(f"    最大: {np.max(bbox_heights):.1f} px")
    print(f"  面積:")
    print(f"    平均: {np.mean(bbox_areas):.1f} px²")
    print(f"    最小: {np.min(bbox_areas):.1f} px²")
    print(f"    最大: {np.max(bbox_areas):.1f} px²")
    
    # 画像に対する割合
    avg_width_ratio = np.mean(bbox_widths) / CANVAS_SIZE * 100
    avg_height_ratio = np.mean(bbox_heights) / CANVAS_SIZE * 100
    avg_area_ratio = np.mean(bbox_areas) / (CANVAS_SIZE * CANVAS_SIZE) * 100
    
    print(f"\n画像に対する割合:")
    print(f"  幅: 平均 {avg_width_ratio:.1f}%")
    print(f"  高さ: 平均 {avg_height_ratio:.1f}%")
    print(f"  面積: 平均 {avg_area_ratio:.2f}%")
    
    # 推奨値との比較
    min_width = np.min(bbox_widths)
    min_height = np.min(bbox_heights)
    min_area_ratio = (min_width * min_height) / (CANVAS_SIZE * CANVAS_SIZE) * 100
    
    print(f"\n⚠️  推奨値との比較:")
    print(f"  最小bboxサイズ: {min_width:.1f}x{min_height:.1f} px ({min_area_ratio:.2f}%)")
    if min_width < MIN_BBOX_SIZE or min_height < MIN_BBOX_SIZE:
        print(f"  ⚠️  警告: 最小bboxが{MIN_BBOX_SIZE}px未満です（学習に影響がある可能性）")
        print(f"     MIN_SCALEを大きくするか、MIN_BBOX_SIZEを調整してください")
    else:
        print(f"  ✓ 最小bboxサイズは{MIN_BBOX_SIZE}px以上です")
    
    if avg_area_ratio < 0.5:
        print(f"  ⚠️  警告: 平均面積が0.5%未満です（bboxが小さすぎる可能性）")
        print(f"     MIN_SCALEを大きくすることを推奨します")
    else:
        print(f"  ✓ 平均面積は0.5%以上です（学習に適したサイズ）")
    
    # サイズ分布のヒストグラム
    print(f"\n📊 サイズ分布:")
    print(f"  幅の分布:")
    print(f"    < 32px: {sum(1 for w in bbox_widths if w < 32)} 件 ({sum(1 for w in bbox_widths if w < 32)/len(bbox_widths)*100:.1f}%)")
    print(f"    32-64px: {sum(1 for w in bbox_widths if 32 <= w < 64)} 件 ({sum(1 for w in bbox_widths if 32 <= w < 64)/len(bbox_widths)*100:.1f}%)")
    print(f"    64-128px: {sum(1 for w in bbox_widths if 64 <= w < 128)} 件 ({sum(1 for w in bbox_widths if 64 <= w < 128)/len(bbox_widths)*100:.1f}%)")
    print(f"    >= 128px: {sum(1 for w in bbox_widths if w >= 128)} 件 ({sum(1 for w in bbox_widths if w >= 128)/len(bbox_widths)*100:.1f}%)")
else:
    print("  ⚠️  bboxデータが見つかりませんでした")

print(f"\n✓ bboxサイズ統計確認完了")


In [ ]:
# ============================================
# 生成データの検証（可視化）
# ============================================

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import random

print("=" * 80)
print("生成データの検証（可視化）")
print("=" * 80)

# ランダムにサンプルを選んで表示
num_samples = 5
generated_images = list(TRAIN_IMAGES_DIR.glob("*.png"))
if len(generated_images) == 0:
    print("⚠️  生成された画像がありません")
else:
    sample_images = random.sample(generated_images, min(num_samples, len(generated_images)))
    
    print(f"\nサンプル表示（{len(sample_images)}件）:")
    print(f"  画像サイズ: {CANVAS_SIZE}x{CANVAS_SIZE} px（実際のサイズで表示）")
    
    # 640x640の画像を適切に表示するためのfigsize設定
    # 1画像あたりのサイズ: 幅8インチ、高さ8インチ（アスペクト比1:1）
    fig_width = 8
    fig_height = 8 * len(sample_images)
    fig, axes = plt.subplots(len(sample_images), 1, figsize=(fig_width, fig_height))
    if len(sample_images) == 1:
        axes = [axes]
    
    for idx, img_path in enumerate(sample_images):
        # 画像を読み込み
        img = Image.open(img_path)
        img_width, img_height = img.size
        
        # 画像サイズを確認（640x640であることを確認）
        if img_width != CANVAS_SIZE or img_height != CANVAS_SIZE:
            print(f"  ⚠️  警告: {img_path.name} のサイズが{CANVAS_SIZE}x{CANVAS_SIZE}ではありません ({img_width}x{img_height})")
        
        # ラベルを読み込み
        label_path = TRAIN_LABELS_DIR / f"{img_path.stem}.txt"
        labels = []
        if label_path.exists():
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        class_id = int(parts[0])
                        center_x = float(parts[1])
                        center_y = float(parts[2])
                        width = float(parts[3])
                        height = float(parts[4])
                        
                        # YOLO形式からピクセル座標に変換
                        x_center = center_x * img_width
                        y_center = center_y * img_height
                        bbox_width = width * img_width
                        bbox_height = height * img_height
                        
                        xmin = x_center - bbox_width / 2
                        ymin = y_center - bbox_height / 2
                        xmax = x_center + bbox_width / 2
                        ymax = y_center + bbox_height / 2
                        
                        class_name = "(" if class_id == CLASS_ID_LPAREN else ")" if class_id == CLASS_ID_RPAREN else f"class_{class_id}"
                        labels.append({
                            'class_id': class_id,
                            'class_name': class_name,
                            'bbox': (xmin, ymin, xmax, ymax)
                        })
        
        # 画像を表示（640x640の実際のサイズで表示）
        ax = axes[idx]
        ax.imshow(img)
        ax.set_xlim(0, img_width)
        ax.set_ylim(img_height, 0)  # Y軸を反転（画像座標系）
        ax.set_aspect('equal')  # アスペクト比を1:1に保つ
        ax.axis('off')
        
        # グリッドを表示（オプション：bboxサイズを確認しやすくする）
        # ax.grid(True, alpha=0.3, linestyle='--')
        
        # バウンディングボックスを描画
        for label in labels:
            class_id = label['class_id']
            class_name = label['class_name']
            xmin, ymin, xmax, ymax = label['bbox']
            
            # バウンディングボックス
            color = 'red' if class_id == CLASS_ID_LPAREN else 'blue'
            rect = patches.Rectangle(
                (xmin, ymin), xmax - xmin, ymax - ymin,
                linewidth=2, edgecolor=color,
                facecolor='none'
            )
            ax.add_patch(rect)
            
            # クラス名を表示
            ax.text(
                xmin, ymin - 5, class_name,
                color=color,
                fontsize=12, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7)
            )
        
        # タイトル（bboxサイズ情報も含める）
        title = f"UUID: {img_path.stem[:20]}... | 検出数: {len(labels)}"
        if len(labels) > 0:
            classes_found = [l['class_name'] for l in labels]
            title += f" | クラス: {', '.join(set(classes_found))}"
            # bboxサイズ情報を追加
            if len(labels) > 0:
                first_bbox = labels[0]['bbox']
                bbox_w = first_bbox[2] - first_bbox[0]
                bbox_h = first_bbox[3] - first_bbox[1]
                bbox_area_ratio = (bbox_w * bbox_h) / (img_width * img_height) * 100
                title += f"\nbboxサイズ: {bbox_w:.0f}x{bbox_h:.0f}px ({bbox_w/img_width*100:.1f}% x {bbox_h/img_height*100:.1f}%, 面積: {bbox_area_ratio:.2f}%)"
                # 推奨値との比較
                if bbox_w < MIN_BBOX_SIZE or bbox_h < MIN_BBOX_SIZE:
                    title += f" ⚠️ 小さすぎる"
                elif bbox_area_ratio < 0.5:
                    title += f" ⚠️ 面積が小さい"
                else:
                    title += f" ✓ 適切"
        ax.set_title(title, fontsize=10, fontweight='bold')
        
        # 画像サイズの情報を表示
        ax.text(10, img_height - 20, f"画像サイズ: {img_width}x{img_height}px", 
                fontsize=9, color='gray', 
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))
    
    plt.tight_layout()
    plt.show()

print(f"\n✓ 検証完了")


## セル7: tar化（S3保管用）


In [ ]:
# ============================================
# tar化（S3保管用）
# ============================================

import tarfile
import time
from tqdm import tqdm

print("=" * 80)
print("tar化（S3保管用）")
print("=" * 80)

# tarファイルの出力先
tar_dir = OUT_DIR.parent / "paren_tars"
tar_dir.mkdir(parents=True, exist_ok=True)

print(f"\n出力先: {tar_dir}")

# tarファイルのパス
train_images_tar = tar_dir / "paren_train_images.tar"
train_labels_tar = tar_dir / "paren_train_labels.tar"

def create_tar(source_dir, tar_path, desc):
    """tarファイルを作成"""
    start_time = time.time()
    print(f"\n📦 {desc}を作成中...")
    
    with tarfile.open(tar_path, 'w') as tar:
        # ディレクトリ内の全ファイルを追加
        files = list(source_dir.rglob('*'))
        for file_path in tqdm(files, desc=desc):
            if file_path.is_file():
                # 相対パスで追加
                arcname = file_path.relative_to(source_dir)
                tar.add(file_path, arcname=arcname)
    
    elapsed = time.time() - start_time
    size_mb = tar_path.stat().st_size / (1024 * 1024)
    print(f"  ✓ 完了: {tar_path.name} ({size_mb:.2f} MB, {elapsed:.1f}秒)")
    return size_mb

# train_images.tar
if not train_images_tar.exists() or FORCE_REBUILD_TAR:
    if train_images_tar.exists() and FORCE_REBUILD_TAR:
        print(f"\n🔄 FORCE_REBUILD_TAR=True のため、既存のtarファイルを再作成します")
        train_images_tar.unlink()
    create_tar(TRAIN_IMAGES_DIR, train_images_tar, "paren_train_images.tar")
else:
    size_mb = train_images_tar.stat().st_size / (1024 * 1024)
    print(f"\n✓ paren_train_images.tar は既に存在: {size_mb:.2f} MB")
    print(f"  ⚠️  再作成する場合は、FORCE_REBUILD_TAR=True に設定してください")

# train_labels.tar
if not train_labels_tar.exists() or FORCE_REBUILD_TAR:
    if train_labels_tar.exists() and FORCE_REBUILD_TAR:
        print(f"\n🔄 FORCE_REBUILD_TAR=True のため、既存のtarファイルを再作成します")
        train_labels_tar.unlink()
    create_tar(TRAIN_LABELS_DIR, train_labels_tar, "paren_train_labels.tar")
else:
    size_mb = train_labels_tar.stat().st_size / (1024 * 1024)
    print(f"\n✓ paren_train_labels.tar は既に存在: {size_mb:.2f} MB")
    print(f"  ⚠️  再作成する場合は、FORCE_REBUILD_TAR=True に設定してください")

# 合計サイズ
total_size = sum(f.stat().st_size for f in [train_images_tar, train_labels_tar] if f.exists())
total_size_mb = total_size / (1024 * 1024)
print(f"\n✓ tar化完了")
print(f"  合計サイズ: {total_size_mb:.2f} MB")
print(f"  出力先: {tar_dir}")


## セル8: S3へアップロード（任意）


In [ ]:
# ============================================
# S3へアップロード（任意）
# ============================================

import subprocess
import time

if not UPLOAD_TO_S3:
    print("⚠️  UPLOAD_TO_S3=False のため、S3アップロードをスキップします")
else:
    print("=" * 80)
    print("S3へアップロード")
    print("=" * 80)
    
    # AWS認証情報の確認
    try:
        result = subprocess.run(["aws", "sts", "get-caller-identity"], 
                              capture_output=True, text=True, check=True)
        print(f"\n✓ AWS認証情報を確認")
    except subprocess.CalledProcessError:
        print("\n❌ エラー: AWS認証情報が設定されていません")
        print("   セル0でAWS認証情報を設定してください")
        raise
    
    tar_files = [
        (train_images_tar, "paren_train_images.tar"),
        (train_labels_tar, "paren_train_labels.tar"),
    ]
    
    uploaded = []
    failed = []
    
    for tar_path, tar_name in tar_files:
        if not tar_path.exists():
            print(f"\n⚠️  スキップ: {tar_name} が見つかりません")
            continue
        
        s3_path = f"s3://{S3_BUCKET}/{S3_PREFIX}/{tar_name}"
        
        # S3に既に存在するかチェック
        s3_exists = False
        if FORCE_UPLOAD_S3:
            try:
                result = subprocess.run(
                    ["aws", "s3", "ls", s3_path],
                    capture_output=True, text=True, check=True
                )
                s3_exists = True
                print(f"\n⚠️  {tar_name} は既にS3に存在します")
                print(f"   S3パス: {s3_path}")
                print(f"   FORCE_UPLOAD_S3=True のため、上書きします")
            except subprocess.CalledProcessError:
                s3_exists = False
        else:
            # FORCE_UPLOAD_S3=False の場合は既存チェックしてスキップ
            try:
                result = subprocess.run(
                    ["aws", "s3", "ls", s3_path],
                    capture_output=True, text=True, check=True
                )
                s3_exists = True
                print(f"\n✓ {tar_name} は既にS3に存在します（スキップ）")
                print(f"   S3パス: {s3_path}")
                print(f"   FORCE_UPLOAD_S3=False のため、アップロードをスキップします")
                continue
            except subprocess.CalledProcessError:
                s3_exists = False
        
        print(f"\n📤 {tar_name} をアップロード中...")
        print(f"   S3パス: {s3_path}")
        
        max_retries = 3
        retry_count = 0
        success = False
        
        while retry_count < max_retries and not success:
            try:
                start_time = time.time()
                result = subprocess.run(
                    ["aws", "s3", "cp", str(tar_path), s3_path],
                    capture_output=True, text=True, check=True
                )
                elapsed = time.time() - start_time
                size_mb = tar_path.stat().st_size / (1024 * 1024)
                print(f"  ✓ 完了: {size_mb:.2f} MB, {elapsed:.1f}秒")
                uploaded.append(tar_name)
                success = True
            except subprocess.CalledProcessError as e:
                retry_count += 1
                if retry_count < max_retries:
                    print(f"  ⚠️  リトライ {retry_count}/{max_retries}...")
                    time.sleep(2)
                else:
                    print(f"  ❌ 失敗: {e.stderr}")
                    failed.append(tar_name)
    
    print(f"\n" + "=" * 80)
    print("アップロード結果")
    print("=" * 80)
    print(f"  成功: {len(uploaded)} 件")
    if uploaded:
        for name in uploaded:
            print(f"    ✓ {name}")
    
    print(f"  失敗: {len(failed)} 件")
    if failed:
        for name in failed:
            print(f"    ❌ {name}")
    
    if len(uploaded) == len(tar_files):
        print(f"\n✓ 全てのファイルのアップロードが完了しました")
        print(f"  S3パス: s3://{S3_BUCKET}/{S3_PREFIX}/")
    else:
        print(f"\n⚠️  一部のファイルのアップロードに失敗しました")
        print(f"  失敗したファイルは手動でアップロードしてください")
